## import 

In [1]:
## Github[dataset.py]: https://github.com/ming024/FastSpeech2/blob/master/dataset.py
## Github[config.LibriTTS.preprocess.yaml]: https://github.com/ming024/FastSpeech2/blob/master/config/LibriTTS/preprocess.yaml
## Github[config.LibriTTS.train.yaml] : https://github.com/ming024/FastSpeech2/blob/master/config/LibriTTS/train.yaml
## Github[utils.tools]: https://github.com/ming024/FastSpeech2/blob/master/utils/tools.py#L265

import torch
import yaml
from torch.utils.data import DataLoader
# from utils.utils import to_device
from tqdm import tqdm

from dataset import *

In [2]:
!pwd

/home/heiscold/fs2


In [3]:
os.path.exists("./config/LibriTTS/preprocess.yaml")

True

## config (from `yaml`); train_config, preprocess_config

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

preprocess_config = yaml.load(open("./config/LibriTTS/preprocess.yaml", "r"), Loader=yaml.FullLoader)
train_config = yaml.load(open("./config/LibriTTS/train.yaml", "r"), Loader=yaml.FullLoader)
print("Done")

cuda:0
Done


## Dataset

In [5]:
## Dataset
train_dataset = Dataset("train.txt", preprocess_config, train_config, sort=True, drop_last=True)
val_dataset = Dataset("val.txt", preprocess_config, train_config, sort=False, drop_last=False)
print("Done")

Done


In [20]:
sample = next(iter(train_dataset))
sample.keys()

dict_keys(['id', 'speaker', 'text', 'raw_text', 'mel', 'pitch', 'energy', 'duration'])

In [29]:
sample['id'], sample['speaker'], sample['text'], sample['raw_text']

('10148_10349_000237',
 13099,
 array([151, 160, 158, 165, 151, 159, 148, 159, 152, 151, 158, 165, 159,
        165, 157, 159, 152, 157, 151, 154, 158, 156, 165, 358, 169, 159,
        152, 169, 159, 159, 152, 154, 157, 165, 159, 163, 159, 151, 157,
        154, 148, 156, 163, 165, 148, 151, 158, 152, 159, 157, 159, 165,
        158, 169, 165, 156, 159]),
 'die gespräche mit den bauern auf dem weit entlegenen felde hatten ihm gezeigt daß sie anfingen sich an ihre neuen verhältnisse zu gewöhnen der alte herbergswirt bei dem er für ein weilchen eingekehrt war um sich zu trocknen')

In [30]:
sample['pitch'].shape, sample['energy'].shape, sample['duration'].shape

((152,), (152,), (152,))

## DataLoader; train_loader, val_loader

In [31]:
train_loader = DataLoader(
                    train_dataset,
                    batch_size=train_config["optimizer"]["batch_size"], ## 16 * 4...?
                    shuffle=True,
                    collate_fn=train_dataset.collate_fn,)
print("Train Loader Done")

val_loader = DataLoader(
                        val_dataset,
                        batch_size=train_config["optimizer"]["batch_size"],
                        shuffle=False,
                        collate_fn=val_dataset.collate_fn,
)
print("Valid Loader Done")

Train Loader Done
Valid Loader Done


In [32]:
train_config["optimizer"]["batch_size"]

16

## Check Sample Batch Shape from train_loader

In [45]:
4%16

4

In [66]:
batch = next(iter(train_loader))

In [67]:
len(batch)

1

In [68]:
len(batch[0])

12

In [69]:
batch[0][0] ## basenames

['10148_10349_002768',
 '10148_10349_004866',
 '10148_10349_004202',
 '10148_10349_001164',
 '10148_10349_002564',
 '10148_10349_002712',
 '10148_10119_001276',
 '10148_10349_002174',
 '10148_10349_009185',
 '10148_10349_005704',
 '10148_10349_002323',
 '10148_10349_003060',
 '10148_10349_003243',
 '10148_10349_008143',
 '10148_10119_000333',
 '10148_10349_000179']

In [70]:
batch[0][1] ## sentences

['du hattest mir doch gar nicht bestimmt gesagt daß du hinfahren wolltest mietest du dir im dorfe pferde so ist das erstens für mich peinlich und zweitens was die hauptsache ist die leute übernehmen das bringen dich aber nicht bis an dein ziel ich habe pferde und wenn du mich nicht kränken willst so benutze sie',
 'bei diesen worten begegneten sich die blicke der beiden brüder und trotz seines steten und gerade jetzt besonders lebhaften wunsches nach einem freundschaftlichen und namentlich ungezwungenen verhältnis zu seinem bruder fühlte ljewin daß es ihm unbehaglich war ihn anzusehen',
 'aber in ihrer eigenen familie erfüllte sie und zwar nicht etwa nur um damit ein beispiel zu geben sondern von ganzem herzen streng alle forderungen der kirche und der umstand daß ihre kinder ungefähr seit einem jahr nicht zum abendmahl gegangen waren beunruhigte sie sehr',
 'sie fuhr mit dem papiermesser über die fensterscheibe und hielt dann seine glatte kalte fläche an ihre wange und lachte beinahe 

In [71]:
batch[0][2] 

array([13099, 13099, 13099, 13099, 13099, 13099, 13099, 13099, 13099,
       13099, 13099, 13099, 13099, 13099, 13099, 13099])

In [ ]:
# Each batch
#### --> basename: [... 16개 ...]
#### --> sentences:  [... 16개 ...]

## from github

In [72]:
ids, raw_texts, speakers, texts, src_lens, max_src_len, mels, mel_lens, max_mel_len, pitches, energies, durations = batch[0]

In [73]:
ids  ### 16

['10148_10349_002768',
 '10148_10349_004866',
 '10148_10349_004202',
 '10148_10349_001164',
 '10148_10349_002564',
 '10148_10349_002712',
 '10148_10119_001276',
 '10148_10349_002174',
 '10148_10349_009185',
 '10148_10349_005704',
 '10148_10349_002323',
 '10148_10349_003060',
 '10148_10349_003243',
 '10148_10349_008143',
 '10148_10119_000333',
 '10148_10349_000179']

In [74]:
raw_texts ### 16

['du hattest mir doch gar nicht bestimmt gesagt daß du hinfahren wolltest mietest du dir im dorfe pferde so ist das erstens für mich peinlich und zweitens was die hauptsache ist die leute übernehmen das bringen dich aber nicht bis an dein ziel ich habe pferde und wenn du mich nicht kränken willst so benutze sie',
 'bei diesen worten begegneten sich die blicke der beiden brüder und trotz seines steten und gerade jetzt besonders lebhaften wunsches nach einem freundschaftlichen und namentlich ungezwungenen verhältnis zu seinem bruder fühlte ljewin daß es ihm unbehaglich war ihn anzusehen',
 'aber in ihrer eigenen familie erfüllte sie und zwar nicht etwa nur um damit ein beispiel zu geben sondern von ganzem herzen streng alle forderungen der kirche und der umstand daß ihre kinder ungefähr seit einem jahr nicht zum abendmahl gegangen waren beunruhigte sie sehr',
 'sie fuhr mit dem papiermesser über die fensterscheibe und hielt dann seine glatte kalte fläche an ihre wange und lachte beinahe 

In [75]:
speakers  ### 16

array([13099, 13099, 13099, 13099, 13099, 13099, 13099, 13099, 13099,
       13099, 13099, 13099, 13099, 13099, 13099, 13099])

In [77]:
texts, texts.shape  ### 16

(array([[151, 154, 163, ..., 148, 159, 169],
        [148, 151, 169, ...,   0,   0,   0],
        [148, 159, 159, ...,   0,   0,   0],
        ...,
        [151, 158, 159, ...,   0,   0,   0],
        [159, 165, 151, ...,   0,   0,   0],
        [157, 163, 160, ...,   0,   0,   0]]),
 (16, 91))

In [78]:
 src_lens, max_src_len,   ### 16

(array([91, 79, 65, 61, 59, 58, 57, 54, 45, 45, 41, 40, 37, 37, 33, 30]), 91)

In [81]:
mels.shape, mel_lens,   ### 16

((16, 1397, 80),
 array([1359, 1345, 1397, 1039,  999, 1253, 1132,  889,  858,  853,  712,
         823,  805,  793,  761,  733]))

In [80]:
max_mel_len, pitches.shape, energies.shape, durations.shape  ### 16

(1397, (16, 208), (16, 208), (16, 208))

#### to_device function from `utils.tools.py`

In [6]:
### utils.tools.py
def to_device(data, device):
    if len(data) == 12:
        (
            ids,
            raw_texts,
            speakers,
            texts,
            src_lens,
            max_src_len,
            mels,
            mel_lens,
            max_mel_len,
            pitches,
            energies,
            durations,
        ) = data

        speakers = torch.from_numpy(speakers).long().to(device)
        texts = torch.from_numpy(texts).long().to(device)
        src_lens = torch.from_numpy(src_lens).to(device)
        mels = torch.from_numpy(mels).float().to(device)
        mel_lens = torch.from_numpy(mel_lens).to(device)
        pitches = torch.from_numpy(pitches).float().to(device)
        energies = torch.from_numpy(energies).to(device)
        durations = torch.from_numpy(durations).long().to(device)

        return (
            ids,
            raw_texts,
            speakers,
            texts,
            src_lens,
            max_src_len,
            mels,
            mel_lens,
            max_mel_len,
            pitches,
            energies,
            durations,
        )

    if len(data) == 6:
        (ids, raw_texts, speakers, texts, src_lens, max_src_len) = data

        speakers = torch.from_numpy(speakers).long().to(device)
        texts = torch.from_numpy(texts).long().to(device)
        src_lens = torch.from_numpy(src_lens).to(device)

        return (ids, raw_texts, speakers, texts, src_lens, max_src_len)


In [54]:
################### Eval ###################### 
n_batch = 0
for batchs in val_loader:
    for batch in batchs:
        # to_device(batch, device)
        n_batch += 1

        
print("Validation set  with size {} is composed of {} batches.".format( len(val_dataset), n_batch ))
## 4m 10.4s @ CPU -- valid dataset.__len__: 512

Validation set  with size 512 is composed of 32 batches.


In [55]:
32 * 16
# N_BATCHES * BATCH_SIZE = 512

512

## Change Batch Size (as example)

In [7]:
trainloader = DataLoader(
                    train_dataset,
                    batch_size=train_config["optimizer"]["batch_size"] * 4, ## 16 * 4...?
                    shuffle=True,
                    collate_fn=train_dataset.collate_fn,)
                    
print("Train Loader Done")

Train Loader Done


In [11]:
############################ Train: SAMPLE TEST ##############################
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_batch = 0

# for batchs in trainloader:
for i, batchs in enumerate( tqdm(trainloader, total = len(trainloader)) ):
    for batch in batchs:
        to_device(batch, device) ## 이래서 Device로 보냈구나...
        n_batch += 1

    if i ==1:
        break
        
print("Training set  with size {} is composed of {} batches.".format( len(train_dataset), n_batch ))
###   1%|          | 1/197 [00:05<18:28,  5.65s/it]
### Training set  with size 12584 is composed of 8 batches.

  1%|          | 1/197 [00:05<18:28,  5.65s/it]

Training set  with size 12584 is composed of 8 batches.


In [12]:
############################ Train: Full ##############################
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_batch = 0

# for batchs in trainloader:
for i, batchs in enumerate( tqdm(trainloader, total = len(trainloader)) ):
    for batch in batchs:
        to_device(batch, device) ## 이래서 Device로 보냈구나...
        n_batch += 1

    # if i ==1:
    #     break
        
print("Training set  with size {} is composed of {} batches.".format( len(train_dataset), n_batch ))

#### 어?? 아깐 이렇게 해도 안 끝났는데?? 뭐지??? 
####   >>>> 22m 10.4s @ GPU -- train_dataset.__len__: (13100 - 512)


######## 뭐지...? 다시 돌리니까 단축이 왜 된 거지? ##########
#### 100%|██████████| 197/197 [20:12<00:00,  6.15s/it]
#### Training set  with size 12584 is composed of 786 batches.

100%|██████████| 197/197 [20:12<00:00,  6.15s/it]

Training set  with size 12584 is composed of 786 batches.


In [13]:
print("Check Finished")

Check Finished
